### Importación de librerías

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy as sp
import sklearn as sk
import itertools as it
import prince
import warnings

# Para una visualización más limpia (esta parte del código se con todo estructurado)
# para evitar ignorar los warnings necesarios durante el proceso de desarrollo
# warnings.filterwarnings("ignore")

# Configuración pandas
pd.set_option('display.max_columns', None)

### Apertura del dataset (resumido en la entrega anterior).

In [3]:
df = pd.read_parquet("GRD_2023_DESCRIPTIVE_CLEAN.parquet", engine = "pyarrow")
df.head(10)

,SEXO,SERVICIO_SALUD,TIPO_INGRESO,FECHA_INGRESO,FECHAALTA,TIPOALTA,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD,EDAD,DIAS_ESTANCIA,HOSPITAL,Descripción,peso_categoria,Grupo Etario
0,HOMBRE,METROPOLITANO SURORIENTE,URGENCIA,2023-11-04,2023-11-10,DOMICILIO,1.0,1.0,22.0,6,Complejo Hospitalario Dr. Sótero del Río (San...,Otros tipos de neumotórax espontáneo,Medio,Adulto
1,HOMBRE,METROPOLITANO SURORIENTE,URGENCIA,2023-06-07,2023-06-08,HOSPITALIZACIÓN DOMICILIARIA,1.0,1.0,3.0,1,Complejo Hospitalario Dr. Sótero del Río (San...,Neumonía debida a virus sincitial respiratorio,Bajo,Niño
2,HOMBRE,METROPOLITANO ORIENTE,PROGRAMADA,2023-08-09,2023-08-10,DOMICILIO,2.0,1.0,11.0,1,Hospital de Niños Dr. Luis Calvo Mackenna (Sa...,Fisura del paladar duro y del paladar blando c...,Medio,Niño
3,MUJER,METROPOLITANO SURORIENTE,OBSTETRICA,2023-08-28,2023-08-30,DOMICILIO,1.0,1.0,23.0,2,Hospital Clínico Metropolitano La Florida Dra...,Parto por cesárea de emergencia,Bajo,Adulto
4,HOMBRE,METROPOLITANO CENTRAL,URGENCIA,2023-07-31,2023-08-04,DOMICILIO,1.0,1.0,47.0,4,Hospital de Urgencia Asistencia Pública Dr. A...,Cálculo de conducto biliar con colangitis,Medio-Bajo,Adulto
5,MUJER,METROPOLITANO SUR,PROGRAMADA,2023-04-13,2023-04-17,DOMICILIO,1.0,1.0,63.0,4,"Hospital Barros Luco Trudeau (Santiago, San M...","Estenosis espinal, región lumbar",Muy Alto,Adulto
6,MUJER,METROPOLITANO OCCIDENTE,OBSTETRICA,2023-08-05,2023-08-07,DOMICILIO,1.0,1.0,30.0,2,Hospital San José (Melipilla),Trabajo de parto y parto complicados por circu...,Bajo,Adulto
7,MUJER,METROPOLITANO NORTE,OBSTETRICA,2023-07-26,2023-08-01,DOMICILIO,1.0,1.0,17.0,6,"Complejo Hospitalario San José (Santiago, Ind...",Atención materna por incompetencia del cuello ...,Medio-Bajo,Adolescente
8,MUJER,METROPOLITANO SURORIENTE,URGENCIA,2023-10-02,2023-10-03,DOMICILIO,1.0,1.0,40.0,1,Hospital Clínico Metropolitano La Florida Dra...,Tumor de comportamiento incierto o desconocido...,Bajo,Adulto
9,HOMBRE,METROPOLITANO SURORIENTE,URGENCIA,2023-03-04,2023-03-07,DOMICILIO,1.0,1.0,1.0,3,Hospital Padre Alberto Hurtado (San Ramón),"Neumonía viral, no especificada",Bajo,Niño


Para evitar colinealidad, eliminamos la edad

In [4]:
df = df.drop(columns=["EDAD"])
df.head()

,SEXO,SERVICIO_SALUD,TIPO_INGRESO,FECHA_INGRESO,FECHAALTA,TIPOALTA,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD,DIAS_ESTANCIA,HOSPITAL,Descripción,peso_categoria,Grupo Etario
0,HOMBRE,METROPOLITANO SURORIENTE,URGENCIA,2023-11-04,2023-11-10,DOMICILIO,1.0,1.0,6,Complejo Hospitalario Dr. Sótero del Río (San...,Otros tipos de neumotórax espontáneo,Medio,Adulto
1,HOMBRE,METROPOLITANO SURORIENTE,URGENCIA,2023-06-07,2023-06-08,HOSPITALIZACIÓN DOMICILIARIA,1.0,1.0,1,Complejo Hospitalario Dr. Sótero del Río (San...,Neumonía debida a virus sincitial respiratorio,Bajo,Niño
2,HOMBRE,METROPOLITANO ORIENTE,PROGRAMADA,2023-08-09,2023-08-10,DOMICILIO,2.0,1.0,1,Hospital de Niños Dr. Luis Calvo Mackenna (Sa...,Fisura del paladar duro y del paladar blando c...,Medio,Niño
3,MUJER,METROPOLITANO SURORIENTE,OBSTETRICA,2023-08-28,2023-08-30,DOMICILIO,1.0,1.0,2,Hospital Clínico Metropolitano La Florida Dra...,Parto por cesárea de emergencia,Bajo,Adulto
4,HOMBRE,METROPOLITANO CENTRAL,URGENCIA,2023-07-31,2023-08-04,DOMICILIO,1.0,1.0,4,Hospital de Urgencia Asistencia Pública Dr. A...,Cálculo de conducto biliar con colangitis,Medio-Bajo,Adulto
